In [179]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

#open Trip Advisor url and transforms it into BeautifulSoup object

def bs(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None

    bsObj = BeautifulSoup(html.read(), "html.parser")
    return(bsObj)

bsObj = bs("https://www.tripadvisor.com.br/Restaurant_Review-g303631-d1063102-Reviews-Famiglia_Mancini-Sao_Paulo_State_of_Sao_Paulo.html")

In [150]:
#encontrar número de avaliações
def aval(bsObj):
    av = bsObj.find("span", {"property":"v:count"})
    avText = av.getText()
    return(float(avText))

aval(bsObj)

10.404

In [151]:
#encontrar número de avaliações
#(2,3) = $$-$$$

def price(bsObj, separator):
    p = bsObj.find("span", {"class":"header_tags rating_and_popularity"})
    pText = p.getText()

    position = pText.find(separator)
    count1 = 0
    count2 = 0

    for char in pText[:position]:
        if char == '$':
            count1 += 1

    for char in pText[position+1:]:
        if char == '$':
            count2 += 1
    
    return((count1, count2))

price(bsObj, '-')

(2, 3)

In [170]:
#encontrar localidade
#rua, cidade/estado//cep, país

def local(bsObj):
    rua = bsObj.find("span", {"class":"street-address"})
    cid_est_cep = bsObj.find("span", {"class":"locality"})
    pais = bsObj.find("span", {"class":"country-name"})
    
    lista = []
    
    try:
        lista.append(rua.getText())
    except AttributeError as e:
        lista.append("erro")
    
    try:
        lista.append(cid_est_cep.getText())
    except AttributeError as e:
        lista.append("erro")
    
    try:
        lista.append(pais.getText())
    except:
        lista.append("erro")

    return(lista)

local(bsObj)

['Rua Avanhandava, 81', 'São Paulo, Estado de São Paulo 01306-001, ', 'Brasil']

In [153]:
#overall rating (numeric)

def rating(bsObj):
    rating = bsObj.find("span", {"class":"overallRating"})
    rating = rating.getText()

    if rating.find(',') != -1:
        position = rating.find(',')
        rating_raw = rating[0:position]+rating[position+1:]
        rating = str(float(rating_raw)/10)
        
    return(float(rating))

rating(bsObj)

4.5

In [154]:
#tipo de cozinha do restaurante

def cuisine(bsObj):
    cuisine = bsObj.findAll("div",{"class":"text"})[1]
    cuisine = cuisine.getText()
       
    return(cuisine)

cuisine(bsObj)

' Italiana, Grelhados '

In [155]:
for child in bsObj.find("div",{"class":"ui_column is-6"}).children:
    if child.find('text') != -1:
        print(child.getText())


Serviço


In [156]:
attributes = []

for bigchild in bsObj.find("div",{"class":"ui_columns is-multiline questionRatings"}):
    if bigchild.find('ui_column is-6') != -1:
        for child in bigchild.children:
            if child.find('text') != -1:
                attributes.append(child.getText())
   
att_clean = [attributes[1],attributes[3],attributes[5],attributes[7]]
att_clean

['Serviço', 'Comida', 'Preço', 'Ambiente']

In [157]:
import pandas as pd
import csv

In [188]:
def xlsxGetUrl(line):
    excel = pd.read_excel(r"C:\Users\Helio\Documents\EESP DATA\Projetos\Oportunidade de Negócios\tripAd.xlsx").values
    url = excel[line][0]
    return url

lista_endereco = []

In [ ]:
size = len(pd.read_excel(r"C:\Users\Helio\Documents\EESP DATA\Projetos\Oportunidade de Negócios\tripAd.xlsx").values)

for i in range(22593, size): #continuar pelo 14951
    bsObj = bs(xlsxGetUrl(i))
    endereco = local(bsObj)
    lista_endereco.append(endereco)
    print(i)

In [1]:
lista_endereco[37049]

NameError: name 'lista_endereco' is not defined

In [404]:
lista_endereco[1][1][-11:][:-2]

'05678-000'

In [401]:
lista_final = []

for i in range(0,len(lista_endereco)):
    lista_final.append((lista_endereco[i][0]))
    
lista_final

['Rua Galeno de Revoredo 22',
 'Avenida Doutor Alberto Penteado 605',
 'Avenida Doutor Guilherme Dumont Villares 1210',
 'Alameda das Boninas 315',
 'Avenida Presidente Juscelino Kubitschek 1197',
 'Rua Borges Lagoa 720',
 'Rua Augusto Severo 97',
 'Rua Treze de Maio 513',
 'Rua Cubatao 405',
 'Rua Caetano Teixeira 360',
 'R Itu,Bsa, 281',
 'Rua Bela Cintra 2325',
 'Rua Tuiuti 2796',
 'Rua Caetano Leme da Cunha Filho 101',
 'Avenida dos Guachos 8',
 'Rua Albuquerque Maranhao 71',
 'Rua Jose Jannarelli, 437',
 'Rua Artur de Azevedo 284',
 'Localizado no Mercado Municipal de Sao Paulo',
 'Rua Henrique Sertorio 252',
 'Rodrigues Barbosa, 232',
 'Rua da Mooca 3135',
 'Rua Araujo 124',
 'Rua Inácio Pereira Da Rocha, 379',
 'Rua Gomes de Carvalho 551',
 'Av. Lins de Vasconcelos, 1353',
 'R Heitor Peixoto, 478',
 'Rua Renato Paes de Barros, 198',
 'Rua Doutor Melo Alves, 238',
 'Alameda Santos, 24',
 'Rua Basilio da Cunha,155',
 'Rua Luis Gois, 1283',
 'Rua Almeria 744',
 'Rua Alexandre Dumas

In [417]:
lista_final[37049]

'Alameda dos Nhambiquaras 1670'

In [439]:
lista_endereco[37032]

['Tv D 4', 'São Paulo, Estado de São Paulo, ', 'Brasil']

In [440]:
with open('EnderecosFinalFull.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        for i, row in enumerate(lista_endereco):
            try:
                writer.writerow([row])
            except:
                writer.writerow(['erro'])